# Model Selection

In diesem Notebook demonstieren wir die `hold-out cross validation` und die `k-fold cross validation` im Code.

Beide Verfahren sind im `sklearn` für uns bereits implementiert:

Die `hold-out cross validation` machen wir mittels der `train_test_split` Funktion.
Die `k-fold cross validation` machen wir mittels der `cross_val_predict` Funktion.

## Setup

Setup Code muss *nicht* verstanden werden.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [2]:
palette = dict(
    Train=sns.color_palette()[0],
    Val=sns.color_palette()[3]
)

def plot_data(ax, data, x, y):
    if 'kind' in data:
        sns.scatterplot(x=data[x], y=data[y], ax=ax, hue=data['kind'], palette=palette)
    else:
        sns.scatterplot(x=data[x], y=data[y], ax=ax)
    ax.set_xlim(0, df[x].max() + 1)
    ax.set_ylim(0, df[y].max() + 500)

def plot_model(ax, data, x, y, model):
    plot_data(ax, data, x, y)
    y_hat = model.predict(data[[x]])
    sns.lineplot(x=data[x], y=y_hat, color='orange', linewidth=3, ax=ax)
    return y_hat

def plot_model_with_errors(ax, df: pd.DataFrame, x: str, y: str, model):
    def plot_error(row):
        # draw line from real point (media_income, media_house_value) to predicted point (media_income, media_house_value_hat)
        ax.plot([row[x], row[x]], [row[y], row['y_hat']], c='red')
    df = df.copy()
    plot_model(ax, df, x, y, model)
    df['y_hat'] = model.predict(df[[x]])
    df.apply(plot_error, axis=1)

In [3]:
from sklearn.model_selection import train_test_split

df_data = pd.read_csv('data/fish.csv')[['Width', 'Weight']].rename(columns={
    'Width': 'width (cm)',
    'Weight': 'weight (g)'
})

print(f"Data Set Size: {df_data.shape}")

Data Set Size: (159, 2)


## Hold-out cross validation

Wir machen folgendes im Code:

1. Wir teilen die Daten `df_data` in Train-Set `df_train` und Validaiton-Set `df_val` auf.
2. Wir trainieren zwei Modelle, eine Lineare Regression `lr_model` und ein Decision Tree `dt_model` jeweils auf dem Train-Set.
3. Evaluieren wir die beiden trainierten Modelle auf dem Validation-Set, also auf den während dem Training ungesehenen Daten.

In [4]:
# 1. Split data into train set and val set

df_train, df_val = train_test_split(df_data, shuffle=True, test_size=0.3) # LR best

print(f"Train Set Size: {df_train.shape}")
print(f"Val Set Size: {df_val.shape}")


# 2. Train a LinearRegression and DecisionTreeRegressor on the train set 

lr_model = LinearRegression()
_ = lr_model.fit(X=df_train[['width (cm)']], y=df_train['weight (g)']) # Modell lernt (Lernphase)

dt_model = DecisionTreeRegressor()
_ = dt_model.fit(X=df_train[['width (cm)']], y=df_train['weight (g)']) # Modell lernt (Lernphase)

# 3. Evaluate the LinearRegression and DecisionTreeRegressor on the val set 

lr_y_val_hat = lr_model.predict(df_val[['width (cm)']])
dt_y_val_hat = dt_model.predict(df_val[['width (cm)']])

y_val = df_val['weight (g)']
print("MSE LinearRegression =", round(mean_squared_error(y_val, lr_y_val_hat)))
print("MSE DecisionTreeRegressor =", round(mean_squared_error(y_val, dt_y_val_hat)))

Train Set Size: (111, 2)
Val Set Size: (48, 2)
MSE LinearRegression = 41406
MSE RandomForestRegressor = 57950


Wir sehen, dass eines der beiden Modelle besser ist auf dem Validation-Set.

Führen wir die Zelle mehrfach aus, sehen wir, dass es sehr zufällig ist, welches Modell besser ist. Dies liegt am zufälligen Split der Daten in `train_test_split` und dem (zu) kleinen Validation Set von 48 Datenpunkten.
Es ist natürlich schlecht, da man nicht abhängig vom Zufall (zufälligen Split) entscheiden möchte, welches Modell besser ist, für welches Modell man sich entscheidet.

Was können wir tun?

Generell kann man das Validation Set vergrössern. Je grösser das Validation Set desto weniger wahrscheinlich, dass ein Modell nur durch Zufall gut auf den ungesehenen Daten ist.

Wir können das Validation Set indirekt vergrössern mit dem k-fold cross validation Verfahren. Mit dem k-fold cross validation Verfahren ist jeder Datenpunkt einmal im Validation-Set.

Speziell bei wenig Daten (unter 1000) sollte man immer k-fold cross validation verwenden.

## k-Fold cross validation

Das `k-fold cross validation` Verfahren ist hier im Code gar nicht mehr sichtbar, da das Verfahren in `cross_val_predict` ausgeführt wird. In den Slides wir das Verfahren erklärt.

In [5]:
from sklearn.model_selection import cross_val_predict

df = df_data
# df = df_data.sample(n=120)  # Extra: Simulate random subsets of data to show robustness of k-Fold cross validation

lr_model = LinearRegression()
dt_model = DecisionTreeRegressor()

lr_y_hat = cross_val_predict(lr_model, X=df[['width (cm)']], y=df['weight (g)'], cv=10)
dt_y_hat = cross_val_predict(dt_model, X=df[['width (cm)']], y=df['weight (g)'], cv=10)

y = df['weight (g)']
print("MSE LinearRegression =", round(mean_squared_error(y, lr_y_hat)))
print("MSE RandomForestRegressor =", round(mean_squared_error(y, dt_y_hat)))

Bemerkung: Wir verwenden hier alle Daten als Validation Set: 48
MSE LinearRegression = 34521
MSE RandomForestRegressor = 57660


Wenn wir die obere Zeile mehrfach ausführen, sehen wir immer die gleiche Evaluation. Dies ist weil es keinen Zufall mehr gibt, da es keinen zufälligen Validation Split mehr gibt.

Um einen realistischen Zufall zu simulieren, kann man die Zeile einkommentieren. Sie wählt zufällig 120 Beispiele aus dem Datensatz aus. Dies soll den Datenbeschaffungsprozess simulieren, welcher zufällig ist. Wir "fangen" sozusagen immer 120 andere Fische.